In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import seaborn as sns
import lightgbm as lgb
import xgboost as xgb
from scipy import stats
        
input_path = Path('/kaggle/input/tabular-playground-series-jan-2021/')

# Read in the data files

In [ ]:
train = pd.read_csv(input_path / 'train.csv', index_col='id')
display(train.head())

In [ ]:
sns.heatmap(train.corr())

In [ ]:
plt.figure(figsize=(10, 10))
sns.boxplot(data=train)

In [ ]:
z = np.abs(stats.zscore(train))
print(z)

In [ ]:
print(len(train))
train = train[(z < 3).all(axis=1)]
print(len(train))

In [ ]:
test = pd.read_csv(input_path / 'test.csv', index_col='id')

In [ ]:
features = train.columns

In [ ]:
for df in [train, test]:
    df['sum']=train.sum(axis=0)
    df['min'] = train[features].min(axis=0)
    df['max'] = train[features].max(axis=0)
    df['mean'] = train[features].mean(axis=0)
    df['median'] = train[features].median(axis=0)
    df['skew'] = train[features].skew(axis=0)
    df['kurtosis'] = train[features].kurtosis(axis=0)
display(test.head())

In [ ]:
submission = pd.read_csv(input_path / 'sample_submission.csv', index_col='id')
display(submission.head())

## Pull out the target, and make a validation split

In [ ]:
target = train.pop('target')
X_train, X_test, y_train, y_test = train_test_split(train, target, train_size=0.90, random_state=42)

In [ ]:
# model = lgb.LGBMRegressor(n_estimators=500, max_depth=11, learning_rate=0.035,subsample=0.7, colsample_bytree=0.7, random_state=42)
params={
    'n_estimators': 300,
 'subsample': 0.7,
 'colsample_bytree': 0.7,
 'eta': 0.035282124803629786,
 'max_depth': 11,
 'min_child_weight': 10,
    'tree_method':'gpu_hist',
    'random_state':42
}
model = xgb.XGBRegressor(**params)
model.fit(X_train, y_train)
preds = model.predict(X_test)
print(mean_squared_error(preds, y_test)**0.5)

In [ ]:
model.fit(train, target)
submission['target'] = model.predict(test)
submission.to_csv('xgb_regression.csv')